In [13]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [14]:
from sklearn.datasets import load_digits
digits = load_digits()
print(digits.data.shape)

(1797, 64)


In [15]:
X = digits['images']
Y = digits['target']
X.shape, Y.shape

((1797, 8, 8), (1797,))

Ponizszy kod filtruje, czyli pozostawia, tylko obrazki (zarowno X i Y) ktore przedstawiaja cyfry 0 i 1.
Upraszczamy problem do klasyfikacji binarnej.

In [16]:
ix = np.isin(Y, [0,1])
X_zeroone = np.compress(ix, X, axis=0)  # axis=1
Y_zeroone = np.compress(ix, Y, axis=0)
X_zeroone.shape

(360, 8, 8)

Uruchomienie ponizszych linijek skutkuje zaladowaniem pelnego zbioru danych o wszystkich 10ciu cyfrach.

In [17]:
#X_zeroone = X
#Y_zeroone = Y

Podzial na testowe i treningowe dane:

In [18]:
# w przypadku pelnego zbioru danych, ustaw takefirst np na 1500
takefirst=300
X_train, X_test = X_zeroone[:takefirst,:], X_zeroone[takefirst:,]
Y_train, Y_test = Y_zeroone[:takefirst], Y_zeroone[takefirst:]

In [19]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((300, 8, 8), (60, 8, 8), (300,), (60,))

In [20]:
image_size = 8
num_labels = len(np.unique(Y))

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
X_train, Y_train = reformat(X_train, Y_train)
X_test, Y_test = reformat(X_test, Y_test)
print('Training set', X_train.shape, Y_train.shape)
print('Test set', X_test.shape, Y_test.shape)

Training set (300, 64) (300, 10)
Test set (60, 64) (60, 10)


W tym punkcie mamy dane przygotowane do dalszej pracy!

### Problem 1 ###
L2 oznacza dodanie kary do loss function, opisującej wielkość wektora wag. W TensorFlow można obliczyć loss L2 dla tensora t, używając nn.l2_loss (t). Odpowiednia wartość regularyzacji powinna poprawić dokładność.
Zacznijmy od regresji logistycznej. 

In [42]:
batch_size = 64
learning_rate = 0.5
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # ZADANIE
  # dane wejsciowe: uzyj metod tf.placeholder do wczytania danych z pojedynczego batcha
  # pamietaj o wymiarach! pierwszy wymiar obu placehoderow powinien przedstawiac wielkosc batcha
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, None), name='dataset')
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, None), name='labels')
  tf_test_dataset = tf.constant(X_test, dtype=tf.float32)
  
  # Parametry modelu:
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]), dtype=tf.float32)
  biases = tf.Variable(tf.zeros([num_labels]), dtype=tf.float32)
  
  # Obliczanie logitow:
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
      logits=logits, labels=tf_train_labels))

  # ZADANIE
  # dodaj do obliczonego wyzej loss skladowa L2 regularyzujaca wagi (weights), uzyj na przyklad
  # https://www.tensorflow.org/api_docs/python/tf/nn/l2_loss
  # pamietaj o wspolczynniku beta, tj beta*L2
  #loss += None
  
  # Definiujemy optimizer jako Gradient Descent, wywolujac na nim metode minimize 
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predykcje na danych treningowych:
  train_prediction = tf.nn.softmax(logits)

  # ZADANIE
  # oblicz predykcje na danych testowych, uzyj tf.nn.softmax; dalej analogicznie jak logits
  #test_prediction = None

In [43]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [44]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # wybierz offset w danych treningowych
    offset = (step * batch_size) % (X_train.shape[0] - batch_size)
    # wygeneruj minibatch.
    batch_data = X_train[offset:(offset + batch_size), :]
    batch_labels = Y_train[offset:(offset + batch_size), :]
    # ZADANIE:
    # przygotuj obiekt feed_dict - slownik zawierajacy dane treningowe i ich labele 
    # kluczem sa placeholdery w grafie, a wartosciami zmienne numpy zawierajace dane z obecnego batcha
    feed_dict = {
        tf_train_dataset: batch_data,
        tf_train_labels: batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      #print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), Y_test))

Initialized
Minibatch loss at step 0: 86.196823
Minibatch accuracy: 0.0%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%


### Problem
Dropout na ukrytej warstwie sieci neuronowej. 
Pamiętaj: Dropout powinien być wprowadzony tylko podczas treningu, a nie ewaluacji, w przeciwnym razie wyniki oceny byłyby stochastyczne. TensorFlow dostarcza nn.dropout(), ale musisz się upewnić, że jest włączony tylko podczas treningu.

#### Model

In [227]:
batch_size = 32
hidden_nodes = 1024
learning_rate = 0.3
beta = 0.005

graph = tf.Graph()
with graph.as_default():

  # wczytywanie danych wg tej samej metody placeholder jak powyzej
  tf_train_dataset = None
  tf_train_labels = None
  tf_test_dataset = tf.constant(X_test, dtype=tf.float32)

  # Placeholder przechowujacy prawdopodobienstwo pozostawienia wagi
  keep_prob = tf.placeholder(tf.float32)
  
  # Parametry modelu
  weights_1 = tf.Variable( tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
  weights_2 = tf.Variable( tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  def forward_prop(input):
    # ZADANIE: 
    # zaimplementuj h1 jako zmienna przechowujaca wynik aktywacji relu, zastosowanej do 
    # produktu macierzowego input i weights_1, do ktorego dodaj biases_1
    # uzyj https://www.tensorflow.org/api_docs/python/tf/nn/relu
    h1 = None
    
    # ZADANIE:
    # dodaj dropout do warstwy ukrytej h1
    # uzyj https://www.tensorflow.org/api_docs/python/tf/nn/dropout
    drop = None
    return tf.matmul(drop, weights_2) + biases_2
    
  logits = forward_prop(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

  # ZADANIE:
  # jak wyzej, dodaj do loss komponent regularyzujacy L2, jako sume L2 z obydwu zbiorow wag weights_*
  # pamietaj o przemnozeniu przez parametr beta
  loss += None
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

#### Uczenie

In [228]:
#train_dataset_restricted = X_train[:600, :]
#train_labels_restricted = Y_train[:600, :]

num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # wybierz offset w danych treningowych
    offset = (step * batch_size) % (Y_train.shape[0] - batch_size)
    # wygeneruj minibatch.
    batch_data = X_train[offset:(offset + batch_size), :]
    batch_labels = Y_train[offset:(offset + batch_size), :]

    # ZADANIE:
    # zaimplementuj dwa slowniki feed_dict obslugujace dropout
    # uwzglednij Placeholder przechowujacy prawdopodobienstwo pozostawienia wagi z modelu powyzej
    # w trybie forward nie chcemy tracic zadnych wag (uwzglednij odpowiednio w parametrze keep_prob!)
    # w trybie uczacym, ustaw prawdopodobienstwo pozostawienia wagi na 0.5
    feed_dict = None
    feed_dict_w_drop = None
    
    # ZADANIE:
    # uruchamiajac trening, podaj wlasciwy slownik zawierajacy prawdopodobienstwo pozostawienia wagi 
    # odpowiednie dla trybu forward
    _, l, predictions = session.run(  [optimizer, loss, train_prediction], feed_dict=None)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict=feed_dict), Y_test))

Initialized
Minibatch loss at step 0: 2221.077637
Minibatch accuracy: 0.0%
Test accuracy: 91.7%
Minibatch loss at step 500: 476215.812500
Minibatch accuracy: 100.0%
Test accuracy: 100.0%
Minibatch loss at step 1000: 106138.757812
Minibatch accuracy: 93.8%
Test accuracy: 100.0%
Minibatch loss at step 1500: 23656.033203
Minibatch accuracy: 100.0%
Test accuracy: 100.0%
Minibatch loss at step 2000: 5273.715820
Minibatch accuracy: 87.5%
Test accuracy: 96.7%
Minibatch loss at step 2500: 1175.853027
Minibatch accuracy: 71.9%
Test accuracy: 95.0%
Minibatch loss at step 3000: 262.543396
Minibatch accuracy: 65.6%
Test accuracy: 98.3%
Minibatch loss at step 3500: 58.937382
Minibatch accuracy: 68.8%
Test accuracy: 96.7%
Minibatch loss at step 4000: 13.515789
Minibatch accuracy: 71.9%
Test accuracy: 96.7%


##### 

## Zadanie pierwsze - tuning

#### 1. przetestuj na obu sieciach następujące zakresy hiper-arametrów:

batch_size = [8, 64, 128]

learning_rate = [0.2, 0.5, 1.0, 1.2]

beta = [0.0005, 0.005, 0.05]

Zapamietaj najlepszą kombinację parametrów!

#### 2. załaduj cały zbiór danych, nie tylko cyfry 0, 1

- Jak zmieniły się wyniki?

- Spróbuj powiększyć i pomniejszyć zbiór wag warstw ukrytych (hidden_nodes) - jak zmieniły się wyniki?

## Zadanie drugie - Tensorboard

Skopiuj do osobnych komórek zawartość Model i Uczenie powyżej. Dodaj do modelu i uczenia logowanie Tensorboard.

https://www.tensorflow.org/get_started/summaries_and_tensorboard

Dodaj do modelu sieci neuronowej logowanie czterech wartości: loss, loss po L2, accuracy na zbiorze testowym, accuracy na zbiorze uczącym.

Wyświetl wykresy tych zmiennych.

Wyświetl graf obliczeniowy.